## Mount elevation telemetry generator

In [1]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

## Go to step 3 if you do not have a mount CSC running

#### 0. set up mount remote

In [2]:
start_time = datetime.now()
script = salobj.Controller("Script", index=42658885)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
mount = salobj.Remote(script.domain, "MTMount")
print(f'time to start is {datetime.now() - start_time} [s]')

time to start is 0:00:13.476669 [s]


In [3]:
await mount.start_task

#### 1. mount status

In [4]:
sstate = await mount.evt_summaryState.aget(timeout=5)
print('mount state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

mount state State.DISABLED 2021-03-24 18:49:36.037885440


#### Mount needs to be prepared
await salobj.set_summary_state(mount, salobj.State.ENABLED)

In [ ]:
await mount.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.angleActual, pd.to_datetime(mountAngle.private_sndStamp, unit='s'))

#### 2. Put Mount into STANDBY

In [ ]:
async def checkMountTel(mount):
    print('-------------------------------------------------------')
    for i in range(3):
        await asyncio.sleep(1.0)
        mountAngle = await mount.tel_elevation.aget(timeout=10.)
        print("mount elevation angle", mountAngle.angleActual, pd.to_datetime(mountAngle.private_sndStamp, unit='s'))

In [ ]:
#await checkMountTel(mount)
await salobj.set_summary_state(mount, salobj.State.STANDBY)
#await checkMountTel(mount)

In [ ]:
await salobj.set_summary_state(mount, salobj.State.OFFLINE)

#### 3. Initiate Mount Controller object

In [5]:
mountCtrl = salobj.Controller(name="MTMount")
await mountCtrl.start_task

In [7]:
start_time =Time.now()
print(start_time)

freq = 1 #Hz
end_time = start_time + timedelta(minutes=780)
print (end_time)
while Time(datetime.now())< end_time:
    await asyncio.sleep(1.0/freq)
    #mountCtrl.tel_elevation.set_put(angleActual=mountAngle.angleActual+dAngle)
    mountCtrl.tel_elevation.set_put(actualPosition=89.5)
    mountCtrl.tel_azimuth.set_put(actualPosition=45.0)

2021-03-25 14:15:04.960785
2021-03-26 03:15:04.960785


CancelledError: 

### Close up

In [ ]:
await mountCtrl.close()

In [ ]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)